# DICOM to BIDS converter

Welcome to this DICOM to BIDS converter!

This converter only handles the following MRI sequences:
    - MPRAGE
    - FLAIR
    - EPI
    - Phase
    - Diffusion weighted imaging

The converter should be used on dicom files coming directly out of the MR machine.

#### /!\ Important /!\ : Run steps 1 and 2 before running anything else

Authors: Maxence Wynen -- maxencewynen@gmail.com
        Colin Vanden Bulcke -- colin.vandenbulcke@uclouvain.be

## 1. Import dependencies

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
from dicom2bids import *
from change_orientation import *
from reorient_phase_unwrapped import *
from corr_header_for_registration import *
print("[INFO] Imports done")
#"C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/WSBIM2243/database"
# Path to dcm2niix.exe converter -- Only for windows(Comment for unix/macOS users) 
#dicom2niix_path = "dcm2niix" 
dicom2niix_path = r"C:\Users\Cristina\Downloads\dcm2niix.exe" #"C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/dcm2niix.exe" # Uncomment for Windows

[INFO] Imports done


## 2. Make BIDSHandler object

In [2]:
# Create BIDSHandler object, root_dir is the path to the BIDS folder's root
bidshandler = BIDSHandler(root_dir=r"C:\Users\Cristina\Documents\deleteme\BIDS_EXAMPLE\BIDS_ROOT", #"/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels",
                          dicom2niix_path = dicom2niix_path)

## 3. Automatically import a DICOM directory into BIDS format

In [3]:
# Path to dicom directory
DICOM_FOLDER = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/DICOM/sub-059/sub-059_ses-01/GEVENOIS_3992098701"
DICOM_FOLDER = r"C:\Users\Cristina\Documents\deleteme\BIDS_EXAMPLE\131-DICOM"
PATIENT_ID = None
SESSION = None

pat_id, session, dicom_series = bidshandler.convert_dicoms_to_bids(dicomfolder = DICOM_FOLDER, 
                                                                   pat_id      = PATIENT_ID,
                                                                   session     = SESSION, 
                                                                   return_dicom_series=True)

print("[INFO] Done")

[INFO] Starting to convert ...
SUBDIR: C:\Users\Cristina\Documents\deleteme\BIDS_EXAMPLE\131-DICOM\131	DIRS: []
[INFO] Converted dicom files to 1 compressed nifti
['FLAIR']
None
DICOM series not recognized: _131_COR_t1_20190723102634_502_
Path: C:/Users/Cristina/Documents/deleteme/BIDS_EXAMPLE/131-DICOM/131
None
DICOM series not recognized: _131_dADC_epi_20190723102634_402_
Path: C:/Users/Cristina/Documents/deleteme/BIDS_EXAMPLE/131-DICOM/131
['MPRAGE']
['DWI']
['FLAIR']
['acq-mag_T2star']
['acq-phase_T2star']
['T1w_Gd']
['T2']
[INFO] Copying dicom folder to sourcedata ...
[INFO] Done


In [5]:
dicom_series = bidshandler.convert_all_dicoms(DICOM_FOLDER)

[INFO] Starting to convert ...
SUBDIR: C:\Users\Cristina\Documents\deleteme\BIDS_EXAMPLE\131-DICOM\131	DIRS: []
[INFO] Converted dicom files to 1 compressed nifti


In [6]:
bidshandler.delete_nii_json_in_dicomdir(dicom_series)

## 4.Optional functionalities

### 4.1. Delete subject

In [ ]:
# Remove a patient from the database
bidshandler.delete_subject("2097")

### 4.2. Delete session

In [ ]:
# Remove a particular session from a particular patient
bidshandler.delete_session("026", "03")

### 4.3 Rename subject

In [ ]:
# Rename a subject in the database
bidshandler.rename_subject(old_id='1000',new_id='2000')

## 5. Image processing tools

### 5.1. Reorient an image

In [ ]:
EPI_magnitude = 'acq-mag_T2star'
EPI_phase = 'acq-phase_T2star'
phase_wrapped = 'phase_WRAPPED'
T2 = 'T2'
T1 = 'T1'
T1_Gd = 'T1w_Gd'
FLAIRstar = 'acq-star_FLAIR'
QSM = 'QSM'

reorient(subject = '059', session = '02', 
         sequence_name = EPI_magnitude + '_matlab', 
         axis = 'z',
         DIR=bids_dir)

In [ ]:
##### CHANGE AFFINE ######
replace_affine_or_header(path_to_img = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-059/ses-01/anat/sub-059_ses-01_acq-phase_T2star_UNWRAPPED.nii.gz", 
                         path_to_target = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-137/ses-01/anat/sub-137_ses-01_acq-mag_T2star.nii.gz", affine=True, header=False)


### 4.5. Reorient phase image after docker unwrapping

In [ ]:
### REORIENT THE UNWRAPPED PHASE IMAGE AFTER DOCKER UNWRAPPING ###
phase_filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-059/ses-01/anat/sub-059_ses-01_acq-phase_T2star_UNWRAPPED.nii.gz"
t2star_filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-059/ses-01/anat/sub-059_ses-01_acq-mag_T2star.nii.gz"

reorient_phase_unwrapped(phase_filename, t2star_filename)

In [ ]:
### REORIENT THE UNWRAPPED PHASE IMAGE AFTER DOCKER UNWRAPPING (second possibility) ###
phase_filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-026/ses-01/anat/sub-026_ses-01_phase.nii.gz"

reorient_phase_unwrapped_2(phase_filename)

### 5.2. Correct header for registration

In [ ]:
### 4.6. Correct header for registration ###
filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-058/ses-02/anat/sub-058_ses-02_acq-phase_T2star.nii.gz"
corr_header_for_registration(filename)

### 5.3. Separate 4D image EPI

In [ ]:
separate_epi(subject='126', session='01', 
             DIR=bids_dir)